<a href="https://colab.research.google.com/github/Remyar12/news_summary/blob/main/news_summarization_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers  rouge-score

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install rouge-score

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import pandas as pd
import transformers
from torch.utils.data import Dataset, DataLoader


from transformers import AutoTokenizer, AutoModel
sentenc_model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(sentenc_model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import os

sum_dir = "drive/MyDrive/mlexperiments/extractivesummarization/" # location to store and load models

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the path to the CSV file
csv_file_path = "/content/test.csv"

# Load the dataset into a pandas DataFrame
try:
    df = pd.read_csv(csv_file_path)
    print("CSV file loaded successfully.")
except FileNotFoundError:
    print(f"File not found: {csv_file_path}")
    df = pd.DataFrame()  # Create an empty DataFrame if the file is not found

# Display the shape of the dataset
print("Original dataset shape:", df.shape)

# Check the contents of the dataset
print(df.head())

# Split the dataset into training and testing sets
if not df.empty:
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Display the shapes of the training and testing sets
    print("Training set shape:", train_df.shape)
    print("Testing set shape:", test_df.shape)
else:
    print("The dataset is empty or not loaded correctly.")


CSV file loaded successfully.
Original dataset shape: (11490, 3)
                                         id  \
0  92c514c913c0bdfe25341af9fd72b29db544099b   
1  2003841c7dc0e7c5b1a248f9cd536d727f27a45a   
2  91b7d2311527f5c2b63a65ca98d21d9c92485149   
3  caabf9cbdf96eb1410295a673e953d304391bfbb   
4  3da746a7d9afcaa659088c8366ef6347fe6b53ea   

                                             article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   
2  Dougie Freedman is on the verge of agreeing a ...   
3  Liverpool target Neto is also wanted by PSG an...   
4  Bruce Jenner will break his silence in a two-h...   

                                          highlights  
0  Experts question if  packed out planes are put...  
1  Drunk teenage boy climbed into lion enclosure ...  
2  Nottingham Forest are close to extending Dougi...  
3  Fiorentina goalkeeper Neto has been linked wit...  
4  Tell-all interview with the reality T

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import time
import cProfile

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len_input, max_len_output):
        self.data = dataframe
        self.articles = self.data['article']
        self.highlights = self.data['highlights']
        self.tokenizer = tokenizer
        self.max_len_input = max_len_input
        self.max_len_output = max_len_output

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = self.articles[idx]
        highlights = self.highlights[idx]

        input_encoding = self.tokenizer.encode_plus(
            article,
            add_special_tokens=True,
            max_length=self.max_len_input,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        target_encoding = self.tokenizer.encode_plus(
            highlights,
            add_special_tokens=True,
            max_length=self.max_len_output,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'target_ids': target_encoding['input_ids'].flatten(),
            'target_attention_mask': target_encoding['attention_mask'].flatten()
        }

# Corrected file paths
train_file = "/content/output.csv.txttrain_bdf.csv"
test_file = "/content/output.csv.txttest_bdf.csv"

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Reset indices
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# Constants
TRAIN_BATCH_SIZE = 4  # Reduce batch size to avoid memory issues
EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_LEN_INPUT = 256
MAX_LEN_OUTPUT = 128
MODEL_NAME = 't5-small'
GRADIENT_ACCUMULATION_STEPS = 4

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Define CustomDataset and DataLoader
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN_INPUT, MAX_LEN_OUTPUT)
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

# Verify DataLoader
data_iter = iter(train_loader)
batch = next(data_iter)
print(batch)

# Define the model
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scaler = GradScaler()

# Define train function
def train(epoch):
    model.train()
    tr_loss = 0
    nb_tr_steps = 0
    start_time = time.time()

    for step, data in enumerate(tqdm(train_loader, 0)):
        input_ids = data['input_ids'].to(device, dtype=torch.long)
        attention_mask = data['attention_mask'].to(device, dtype=torch.long)
        target_ids = data['target_ids'].to(device, dtype=torch.long)
        target_attention_mask = data['target_attention_mask'].to(device, dtype=torch.long)

        optimizer.zero_grad()

        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids)
            loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS

        scaler.scale(loss).backward()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()

        tr_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS
        nb_tr_steps += 1

        if step % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"Step {step}, Loss: {tr_loss / nb_tr_steps}, Elapsed Time: {elapsed_time:.2f}s")
            start_time = time.time()

    print(f'Epoch {epoch}, Loss: {tr_loss / nb_tr_steps}')

# Profile the training function
cProfile.run('train(0)', sort='cumtime')

# Train the model
for epoch in range(EPOCHS):
    train(epoch)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'input_ids': tensor([[   37,  4627,    13,  ...,   280, 14131,     1],
        [ 7138,   588,    65,  ...,   752,   119,     1],
        [   94,    19,     8,  ...,    12,   217,     1],
        [   71,  2386,  1015,  ...,   894,    12,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'target_ids': tensor([[   37,  4627,    13,     8,   301, 13662,    31,     7, 15039, 10348,
             6,    46, 11989,    18,  6259,    18,  2961, 16455,     3,     7,
          3995,   106,    49,     6,   130,     3, 28540,    57,     8,  5458,
         12899,   778,  2089,  1379,     3,     5, 16233,   724,   141,    12,
          4418,    45,     8,  4383,    12,  2794,  5458, 12899, 12285,     3,
             5, 12020,  1533, 13863,   109,   710,    15,    26,     8,  4418,
            11, 10056,   112,   819,   581,     8,   596,    13,     8,  4383,
             6,   865,   271,

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

Step 2100, Loss: 1.441599176419116, Elapsed Time: 16.09s


 96%|█████████▌| 2202/2298 [05:52<00:13,  7.16it/s]

Step 2200, Loss: 1.4442218920416532, Elapsed Time: 15.87s


100%|██████████| 2298/2298 [06:07<00:00,  6.24it/s]

Epoch 2, Loss: 1.4446985777872765


In [ ]:
import os
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
from rouge_score import rouge_scorer

class CustomTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length_input, max_length_output):
        self.data = dataframe.reset_index(drop=True)  # Reset indices
        self.articles = self.data['article']
        self.highlights = self.data['highlights']
        self.tokenizer = tokenizer
        self.max_length_input = max_length_input
        self.max_length_output = max_length_output

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = str(self.articles[idx])
        highlight = str(self.highlights[idx])

        input_encoding = self.tokenizer.encode_plus(
            article,
            add_special_tokens=True,
            max_length=self.max_length_input,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer.encode_plus(
            highlight,
            add_special_tokens=True,
            max_length=self.max_length_output,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = input_encoding['input_ids'].flatten()
        attention_mask = input_encoding['attention_mask'].flatten()
        labels = target_encoding['input_ids'].flatten()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'highlight': highlight  # Keep original highlight for evaluation
        }

def validate_model(model, dataloader, tokenizer, max_length_output):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    all_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            highlights = batch['highlight']

            with autocast():
                outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length_output)

            # Decode the generated outputs and labels
            generated_summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
            reference_summaries = [highlight for highlight in highlights]

            for gen_sum, ref_sum in zip(generated_summaries, reference_summaries):
                scores = scorer.score(ref_sum, gen_sum)
                for key in scores:
                    all_scores[key].append(scores[key].fmeasure)

    # Calculate average ROUGE scores
    avg_scores = {key: sum(scores) / len(scores) for key, scores in all_scores.items()}
    return avg_scores

# Load your test dataset
test_df = pd.read_csv("/content/output.csv.txttest_bdf.csv")

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Use a smaller subset for quick testing
test_subset_df = test_df.sample(n=100, random_state=42)

# Debugging: Print the first few rows of the subset
print(test_subset_df.head())

# Initialize your dataset
test_dataset = CustomTextDataset(test_subset_df, tokenizer, max_length_input=128, max_length_output=64)  # Reduced lengths

# Debugging: Check the length of the dataset
print("Length of test_dataset:", len(test_dataset))

# Create DataLoader for the test dataset
testing_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)  # Adjusted parameters

# Debugging: Check the first batch
for batch in testing_loader:
    print(batch)
    break

# Load pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Validate the model using the testing_loader
rouge_scores = validate_model(model, testing_loader, tokenizer, max_length_output=64)  # Reduced max_length_output
print("ROUGE scores on test data:", rouge_scores)


                                            id  \
1085  b1585b611a9007d7b86851b6a5c9d32a47040aa8   
2296  f7d9f33ac38ce839c81c7188d33c4e09ce11094e   
208   da39f68c5d9a413e0759fc6bff8260f76ed1ac59   
1178  84bdd724457a4c2161844fb4ff91fefa5465e9aa   
282   b3a62e1415b13dfe0938e206d6353bc57dbd354a   

                                                article  \
1085  It's truly squeaky bum time in the Premier Lea...   
2296  Forget fetching sticks. This musically-minded ...   
208   Martin 'Mad Dog' Allen returns with his latest...   
1178  Jeremy Scott’s spring/summer 2015 collection f...   
282   Joe beats Sachin to 2,000 . Joe Root became th...   

                                             highlights  
1085  Just nine points separates the bottom seven cl...  
2296  Sadie the German Shepherd was filmed at home i...  
208   Martin Allen believes it's a crucial time for ...  
1178  Olivia Phillips, 30, rocked head-to-toe Moschi...  
282   Ben Stokes' dad Gerard tweets his approval of ..

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'input_ids': tensor([[   94,    31,     7,  1892,     3,     7,   835,     9,  3781,  8524,
            51,    97,    16,     8,  6552,  3815,     3,    60,  5772,   257,
          3392,    38,   131,  4169,   979,  2450,     7,     8,  2007,  2391,
          2323,     5,  5716,  1963,   987,     7,   128,    13,     8,  5903,
            16,    11,   859,     8,  1782,  2870,   125,    79,   473,    19,
           831,    21,   135,    12,  1792,     8,  2328,    48,   774,     5,
         11860,    10,     3,    31,  5680,    56,    34,   240,    21,    39,
          1886,    12,  1049,    16,     8,  6552,  3815,    58,    31,  9302,
         13431, 12118,  4132,   476,    41,  2247,  5554,    61,     3,    31,
          5801,    62,  1369,   386,  1031,    27,   317,    62,    56,  1049,
            95,     5,    94,   429,   240,   705,     5,    27,    31,    51,
            59,  2145,    24,    31,     7,     8, 20374,   866,     3,    18,
            68,    62,   310,   174,  

Validating: 100%|██████████| 25/25 [00:38<00:00,  1.55s/it]

ROUGE scores on test data: {'rouge1': 0.3360403885630135, 'rouge2': 0.14025306216838215, 'rougeL': 0.23710997433023764}


In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Assume you have a trained model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Define training function (example)
def train_model(model, train_loader):
    # Training loop here
    pass

# Train your model (example)
# train_model(model, train_loader)

# Save the model
model_save_path = "/content/minilm_bal_exsum.pth"
torch.save(model.state_dict(), model_save_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [2]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.112s


In [3]:
! pip install streamlit transformers torch newspaper3k sumy



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6

In [4]:
!pip install streamlit


In [5]:
!pip install sumy


In [6]:
!pip install newspaper3k


In [7]:
!pip install transformers torch


In [8]:
! pip install streamlit -q

In [9]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.218s


   ╭────────────────────────────────────────────────────────────────╮
   │                                                                │
   │      New major version of npm available! 6.14.8 → 10.8.1       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v10.8.1   │
   │               Run npm install -g npm to update!                │
   │                                                                │
   ╰────────────────────────────────────────────────────────────────╯



In [10]:
!wget -q -O - ipv4.icanhazip.com

34.105.16.144


In [11]:
!pip install nltk

In [14]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import newspaper
import nltk
nltk.download('punkt')


model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.load_state_dict(torch.load("minilm_bal_exsum.pth"))
model.eval()

def summarize_news(text, num_sentences=2):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=num_sentences*20, min_length=num_sentences*10, length_penalty=2.0, num_beams=4, early_stopping=True)
    summarized_news = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summarized_news

def scrape_and_summarize(url, num_sentences=2):
    article = newspaper.Article(url)
    article.download()
    article.parse()
    text = article.text
    summarized_news = summarize_news(text, num_sentences)
    return summarized_news

def main():
    st.title("NEWS SUMMARIZATION APP")
    choice = st.radio("Choose Input Type", ("Enter News", "Enter URL"))
    num_sentences = st.slider("Number of Sentences in Summary:", min_value=1, max_value=3, value=2)

    if choice == "Enter News":
        text = st.text_area("Enter News:")
        if st.button("Summarize News"):
            if text:
                summarized_news = summarize_news(text, num_sentences)
                st.subheader("Summary:")
                st.write(summarized_news)
            else:
                st.warning("Please enter the news to summarize.")

    elif choice == "Enter URL":
        url = st.text_input("Enter URL of the News Article:")
        if st.button("Summarize URL"):
            if url:
                summarized_news = scrape_and_summarize(url, num_sentences)
                st.subheader("Summary:")
                st.write(summarized_news)
            else:
                st.warning("Please enter the URL of the news article.")

if __name__ == "__main__":
    main()


Overwriting app.py


In [15]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.16.144:8501

npx: installed 22 in 1.557s
your url is: https://blue-ads-find.loca.lt
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
tokenizer_config.json: 100% 2.32k/2.32k [00:00<00:00, 13.2MB/s]
spiece.model: 100% 792k/792k [00:00<00:00, 22.1MB/s]
tokenizer.json: 100% 1.39M/1.39M [00:00<00:00, 4.34MB/s]
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading packa